In [ ]:
import os
import pickle 
import numpy as np
import pandas as pd
import re


In [ ]:
output_dir="../../data/train_sentence_pkl"
pkl_files = [fname for fname in os.listdir(output_dir) if fname.startswith("sentence_") and fname.endswith(".pkl")]
def natural_key(s):
    return [int(text) if text.isdigit() else text.lower() for text in re.split('(\d+)', s)]
pkl_files.sort(key=natural_key)
print(pkl_files)


### 파일 크기

In [ ]:
for file_path in sorted(pkl_files):
    size = os.path.getsize(os.path.join(output_dir, file_path))
    print(f"파일 크기: {size} 바이트")

### 저장 형태 확인

In [ ]:
with open("../../data/test_sentence_pkl/sentence_1.pkl", "rb") as f:
    sentence_1=pickle.load(f)


In [ ]:
sentence_1.head(1)

In [ ]:
sentence_1.dtypes

### 전체 데이터 개수 및 title 연속 여부

In [ ]:
from pprint import pprint
data_points=[]
titles=[]
paragraph=[]

nan_list=[]
for idx in range(1,18):
    with open(f"../../data/train_sentence_pkl/sentence_{idx}.pkl", "rb") as f:
        sentence=pickle.load(f)
        # data points
        titles.append(sentence["title"].nunique())
        paragraph.append(sentence.groupby(['title', 'paragraph_index']).ngroups)
        data_points.append(len(sentence))
        # title 연속 여부
        block_titles = sentence['title'][sentence['title'] != sentence['title'].shift()].tolist()
        unique_titles = sentence['title'].unique().tolist()
        if sorted(block_titles) == sorted(unique_titles):
            print("✅ 모든 title 연속 ⭕️")
        else:
            print("⚠️  모든 title 연속 ❌")
        # 결측치
        nan_list.append(sentence.isnull().values.any())
        # pprint(sentence.groupby("generated")["lof"].describe())

In [ ]:
print(f"title: {sum(titles)}")
print(f"paragraph: {sum(paragraph)}")
print(f"total(sentence): {sum(data_points)}")

### 결측치

In [ ]:
if any(nan_list):
    print("✅ 결측치 없음")
else:
    print("⚠️ 결측치 존재")

In [ ]:
# emb_shape
for title, group in sentence_1.groupby('title', sort=False):
    # print(f"title: {title}")
    # print(f"group: {group}")
    emb_matrix = np.vstack(group['sentence_emb'].values)
    # print(emb_matrix)
    print(f"emb_matrix.shape: {emb_matrix.shape}")

    break